# Convert coordinates to bng

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import logging

import sys

sys.path.append("/app")

# import scraping as sc

import pandas as pd

from jinja2 import Template

from db_utils import get_engine, get_table_creation_query

from gis_utils import convert_bng
import os

<IPython.core.display.Javascript object>

In [3]:
# global logger
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter("%(asctime)s [%(name)s] %(levelname)-8s %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

## Connect to Database

In [4]:
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")

<IPython.core.display.Javascript object>

In [5]:
engine = get_engine(user, password, host)

<IPython.core.display.Javascript object>

## Define search parameters

In [6]:
searchname = "tamzin"

<IPython.core.display.Javascript object>

## Load data

In [7]:
q_load = f"""SELECT g.* FROM 
{searchname}.geocoded_addresses g
INNER JOIN
{searchname}.address_ids_to_process a
ON g.address_id=a.address_id"""

<IPython.core.display.Javascript object>

In [8]:
with engine.connect() as conn:
    df = pd.read_sql(q_load, con=conn)

<IPython.core.display.Javascript object>

## Perform Conversion

In [9]:
gdf_bng = convert_bng(df)

2021-05-21 11:20:49,458 [gis_utils] INFO     Converting points to GeoDataFrame...
2021-05-21 11:20:49,479 [gis_utils] INFO     Converting crs...
2021-05-21 11:20:49,619 [gis_utils] INFO     Getting bng coordinates


<IPython.core.display.Javascript object>

## Create table

In [10]:
cols = {
    "eastings": "DECIMAL(14,6)",
    "northings": "DECIMAL(14,6)",
    "address_id": "INTEGER",
}

index_cols = ["address_id"]
unique_cols = ["address_id"]

<IPython.core.display.Javascript object>

In [11]:
create_q = get_table_creation_query(
    "bng_coords", cols, searchname, index_cols, unique_cols
)

<IPython.core.display.Javascript object>

In [12]:
with engine.connect() as conn:
    conn.execute(create_q)

<IPython.core.display.Javascript object>

## Save results

In [13]:
with engine.connect() as conn:
    gdf_bng[["address_id", "eastings", "northings"]].to_sql(
        "bng_coords", schema=searchname, index=False, if_exists="append", con=conn
    )

<IPython.core.display.Javascript object>